Now combining the hovmoller plots for the two different sets of time periods into a single joint figure, following the previous template used in banded_and_local_changes.

In [1]:
from bootstrap import *
from chr_matplotlib import remappedColorMap
from jd_matplotlib_tools import add_double_calendar_axis, mark_time_periods
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import numpy.ma as ma
import os
from shutil import copy
import time

%load_ext autotime

In [2]:
ferretpath = "/Users/Siwen/Desktop/ferret/bin/"
ff = "Pchina_type_notaiwan.nc"
filename = ferretpath + ff
myf = nc.Dataset(filename, 'r')

time: 6.96 ms


In [3]:
P_total =  myf.variables['Pchina_total'][:]
P_band = myf.variables['Pchina_band'][:]
P_local = myf.variables['Pchina_local'][:]

time: 87.1 ms


In [4]:
#years of interest - start and end date, with index starting at 1 out of habit.
p1_yr_b = [1,30]
p1_yr_e = [29,43]
p2_yr_b = [30,44]
p2_yr_e = [57,57]

P_total_diff = []
P_band_diff = []
P_local_diff = []

#cycle through all time periods of interest and find changes in each type of rainfall b/w periods
for p1_b, p1_e, p2_b, p2_e, i in zip(p1_yr_b, p1_yr_e, p2_yr_b, p2_yr_e, np.arange(len(p1_yr_b))): 
    
    P_total_diff.append(np.mean(P_total[:,:,p2_b-1:p2_e], axis=2) - \
                         np.mean(P_total[:,:,p1_b-1:p1_e], axis=2))
    P_band_diff.append(np.mean(P_band[:,:,p2_b-1:p2_e], axis=2) \
                       - np.mean(P_band[:,:,p1_b-1:p1_e], axis=2))
    P_local_diff.append(np.mean(P_local[:,:,p2_b-1:p2_e], axis=2) \
                        - np.mean(P_local[:,:,p1_b-1:p1_e], axis=2))

time: 20.9 ms


In [5]:
def smooth(P,dayrange,latrange):
    Psmooth=np.zeros(P.shape)
    
    ll=int((dayrange-1)/2)
    yy=int((latrange-1)/2)
    
    days = P.shape[0]
    lats = P.shape[1]
    
    for d in range(days):
        for j in range(lats):
            
            Psample = P.take(range(d-ll,d+ll+1), mode='wrap', axis=0)
            Ps = Psample[:,max(0,j-yy):min(days,j+yy+1)]
            Psmooth[d,j] = np.mean(np.mean(Ps))
    
    return Psmooth

time: 6.54 ms


In [6]:
## SMOOTHING ##
#smooth out the decadal precipitation means
#but, no latitudinal smoothing since rainfall already effectively smoothed in latitude
daysmth = 15
latsmth = 0

#one line! thank you python!
P_total_diff_smth = [ smooth(x,daysmth,latsmth) for x in P_total_diff ]
P_band_diff_smth = [ smooth(x,daysmth,latsmth) for x in P_band_diff ]
P_local_diff_smth = [ smooth(x,daysmth,latsmth) for x in P_local_diff ]

time: 4.04 s


In [7]:
## LOAD COMPUTED P-VALUES ##

#which set of sim values are we trying to access?
pval_root = 'Data/chinarain_diff_pval_notaiwan_'
blklen = 2
niter = 2000

P_total_diff_pval = []
P_band_diff_pval = []
P_local_diff_pval = []

#cycle through all time periods of interest and find changes in each type of rainfall b/w periods
for p1_b, p1_e, p2_b, p2_e, i in zip(p1_yr_b, p1_yr_e, p2_yr_b, p2_yr_e, np.arange(len(p1_yr_b))): 
    
    #LOAD P-VALUES FOR TOTAL RAINFALL
    raintype = 'total'
    yrs = str(p1_b+50) + str(p1_e+50) + '_' + str(p2_b+50) + str(p2_e+50) + '_'
    pval_filename = pval_root + yrs + raintype + '_' + str(blklen) + 'day_' \
        + str(niter) + 'iter.nc'
    
    if os.path.isfile(pval_filename):
        print(pval_filename + ' successfully found')
        myf = nc.Dataset(pval_filename, 'r')
        P_total_diff_pval.append(myf.variables['pval'][:])
        myf.close()
        
    else:
        print(pval_filename + ' does not exist')
    
    
    ##LOAD P-VALUES FOR BANDED RAINFALL
    raintype = 'band'
    yrs = str(p1_b+50) + str(p1_e+50) + '_' + str(p2_b+50) + str(p2_e+50) + '_'
    pval_filename = pval_root + yrs + raintype + '_' + str(blklen) + 'day_' \
        + str(niter) + 'iter.nc'
    
    if os.path.isfile(pval_filename):
        print(pval_filename + ' successfully found')
        myf = nc.Dataset(pval_filename, 'r')
        P_band_diff_pval.append(myf.variables['pval'][:])
        myf.close()
        
    else:
        print(pval_filename + ' does not exist')
            
    ##LOAD P-VALUES FOR LOCAL RAINFALL
    raintype = 'local'
    yrs = str(p1_b+50) + str(p1_e+50) + '_' + str(p2_b+50) + str(p2_e+50) + '_'
    pval_filename = pval_root + yrs + raintype + '_' + str(blklen) + 'day_' \
        + str(niter) + 'iter.nc'
    
    if os.path.isfile(pval_filename):
        print(pval_filename + ' successfully found')
        myf = nc.Dataset(pval_filename, 'r')
        P_local_diff_pval.append(myf.variables['pval'][:])
        myf.close()
        
    else:
        print(pval_filename + ' does not exist')    
        
    #P_total_diff.append()
    #P_band_diff.append(np.mean(P_band[:,:,p2_b-1:p2_e], axis=2) \
    #P_local_diff.append(np.mean(P_local[:,:,p2_b-1:p2_e], axis=2) \

Data/chinarain_diff_pval_notaiwan_5179_80107_total_2day_2000iter.nc successfully found
Data/chinarain_diff_pval_notaiwan_5179_80107_band_2day_2000iter.nc successfully found
Data/chinarain_diff_pval_notaiwan_5179_80107_local_2day_2000iter.nc successfully found
Data/chinarain_diff_pval_notaiwan_8093_94107_total_2day_2000iter.nc successfully found
Data/chinarain_diff_pval_notaiwan_8093_94107_band_2day_2000iter.nc successfully found
Data/chinarain_diff_pval_notaiwan_8093_94107_local_2day_2000iter.nc successfully found
time: 70.7 ms


In [8]:
## CREATE MASKED ARRAYS - only show values that are statistically significant
#create two masked arrays - one showing 95% confidence, one showing 99% confidence

##two separate for loops: first one creates masks, second one produces masked rainfall.
#reason for doing things this way is that masks can then be accessed independently at a later point in time.

#not super-pythonic method, but at least it makes it easy to understand.

mk_total = [] #storage format will be a LIST of 2-element tuples - each set of tuples is the 95% and 99% mask respectively
mk_band = []
mk_local = []

#set up 95th and 99th percentile masks for each type of rainfall
for P_total_diff, P_band_diff, P_local_diff, P_total_pval, P_band_pval, P_local_pval in \
    zip(P_total_diff_smth, P_band_diff_smth, P_local_diff_smth, P_total_diff_pval, P_band_diff_pval, P_local_diff_pval):
    
    mk_total_95 = abs(P_total_pval-.5) < .475 #95th percentile mask for rainfall changes
    mk_total_99 = abs(P_total_pval-.5) < .495 #99th percentile mask for rainfall changes
    mk_total.append([mk_total_95, mk_total_99])
    
    mk_band_95 = abs(P_band_pval-.5) < .475 #95th percentile mask for rainfall changes
    mk_band_99 = abs(P_band_pval-.5) < .495 #99th percentile mask for rainfall changes
    mk_band.append([mk_band_95, mk_band_99])
    
    mk_local_95 = abs(P_local_pval-.5) < .475 #95th percentile mask for rainfall changes
    mk_local_99 = abs(P_local_pval-.5) < .495 #99th percentile mask for rainfall changes
    mk_local.append([mk_local_95, mk_local_99])
    
#create masked arrays
P_total_diff_mask = []
P_band_diff_mask = []
P_local_diff_mask = []

#create masked arrays with significant rainfall changes only based on 95th percentile masks
for P_total_diff, P_band_diff, P_local_diff, mask_total, mask_band, mask_local in \
    zip(P_total_diff_smth, P_band_diff_smth, P_local_diff_smth, mk_total, mk_band, mk_local):
    
    P_total_diff_mask.append(ma.array(P_total_diff, mask = mask_total[0]))
    P_band_diff_mask.append(ma.array(P_band_diff, mask = mask_band[0]))
    P_local_diff_mask.append(ma.array(P_local_diff, mask = mask_local[0]))

time: 18.1 ms


In [9]:
## LOAD 1980-2007 v 1951-1979 changes FROM NETCF ##

## NETCDF FILENAMES
freq_diff_file = "RDA_freq_diff_tau_seasonal"
freq_diff_name = ferretpath + freq_diff_file + '.nc'

int_diff_file = "RDA_int_diff"
int_diff_name = ferretpath + int_diff_file + '.nc'

int_pval_file = "RDA_int_diff_bootstrap_2000iter_perm"
int_pval_name = ferretpath + int_pval_file + '.nc'



#import from netCDF file and load 57-year climatology
freq_diff_netcdf = nc.Dataset(freq_diff_name, "r")
int_diff_netcdf = nc.Dataset(int_diff_name, "r")
int_pval_netcdf = nc.Dataset(int_pval_name, "r")

#limits of different time periods that we're comparing
p1_beg = [1951,1980]
p1_end = [1979,1993]
p2_beg = [1980,1994]
p2_end = [2007,2007]

#in this case, we're just going to load the smoothed plots of hovmoller frequency
freq_diff = []
freq_pval = []
int_diff = []
int_pval = []

for p1_b, p1_e, p2_b, p2_e in zip(p1_beg, p1_end, p2_beg, p2_end):

    years_p1= str(p1_b-1900) + str("{0:0=2d}".format((p1_e-1900)%100))
    print(years_p1)
    years_p2= str(p2_b-1900) + str("{0:0=2d}".format((p2_e-1900)%100))
    print(years_p2)
    
    freq_diff.append(freq_diff_netcdf['lat_diff_tau_' + years_p2 + '_' + years_p1][:])
    freq_pval.append(freq_diff_netcdf['lat_pval_tau_' + years_p2 + '_' + years_p1][:])
    int_diff.append(int_diff_netcdf['int_diff_' + years_p2 + '_' + years_p1][:])
    int_pval.append(int_pval_netcdf['int_pval_' + years_p2 + '_' + years_p1][:])
    
#close it
freq_diff_netcdf.close()
int_diff_netcdf.close()
int_pval_netcdf.close()


5179
8007
8093
9407
time: 61 ms


In [10]:
## USE P-VALS TO CREATE MASKS APPLIED TO ONLY SHOW SIGNIFICANT DIFFERENCES
## 1980-2007 versus 1951-1979 - clearly cuts out a lot of random noise!

## frequency masking by analytic p-value
freq_diff_mask_95 = []
freq_diff_mask_99 = []

for fq, pval in zip(freq_diff, freq_pval):
    freq_diff_mask_95.append(ma.array(fq, mask = ~(abs(pval-.5) > .475)))
    freq_diff_mask_99.append(ma.array(fq, mask = ~(abs(pval-.5) > .495)))

#intensity masking by bootstrapped p-value
int_diff_mask_95 = []
int_diff_mask_99 = []   
    
for ints, pval in zip(int_diff, int_pval):
    int_diff_mask_95.append(ma.array(ints, mask = ~(abs(pval-.5) > .475)))
    int_diff_mask_99.append(ma.array(ints, mask = ~(abs(pval-.5) > .495)))

time: 9.35 ms


In retrospect, the way that I did the rest of this code is not up to par, but hindsight is 20/20

In [11]:
## Set up variables that will be plotted in correct order
P_plot_tuples = {'a':[P_total_diff_mask[0].T,P_band_diff_mask[0].T,100*freq_diff_mask_95[0][:,1:21].T,\
                   int_diff_mask_95[0][:,1:21].T],
                 'b':[P_total_diff_mask[1].T,P_band_diff_mask[1].T,100*freq_diff_mask_95[1][:,1:21].T,\
                   int_diff_mask_95[1][:,1:21].T]}

P_contour_tuples = {'a':[mk_total[0][1].T,mk_band[0][1].T,freq_diff_mask_99[0][:,1:21].mask.T,\
                         int_diff_mask_99[0][:,1:21].mask.T],
                    'b':[mk_total[1][1].T,mk_band[1][1].T,freq_diff_mask_99[1][:,1:21].mask.T,\
                         int_diff_mask_99[1][:,1:21].mask.T]}

time: 6.87 ms


In [12]:
## New version of Hovmoller plots of difference, based off of banded_and_local_changes
axlim = 2
mypalette = 'bwr'
yticklabels = ["20$\!^\circ\!$N","24$\!^\circ$","28$\!^\circ$","32$\!^\circ$","36$\!^\circ$","40$\!^\circ\!$N"]
ytickpos = np.linspace(0,80,6)

## Define time periods of interest - these will be demarcated on the plots with vertical lines
time_periods = [[80.5,120.5],[120.5,160.5],[160.5,200.5],[200.5,273.5],[273.5,320.5]]

fig = plt.figure(figsize=(20,10), facecolor='w')
#st = plt.suptitle("Changes in Rainfall by Type", size = 32, y=1, x=.515)

## create grids for left-hand and right-hand sides
grid_left = gridspec.GridSpec(4, 1)
grid_left.update(left=0.05, right=0.48, hspace = .6)
grid_right = gridspec.GridSpec(4, 1)
grid_right.update(left=0.55, right=0.98, hspace = .6)
grids = {'a':grid_left, 'b':grid_right}

## create storage dict for axes objects for each side; a is left, b is right
axes = {'a':[], 'b':[]}

## Define title for each set of subplots; first row a-b, 2nd c-d and so forth
subplot_titles={'a':["a) Change in Total Rainfall (mm day$^{-1}$)","c) Change in Banded Rainfall (mm day$^{-1}$)",\
                "e) Change in Band Frequency (%)","g) Change in Band Intensity (mm day$^{-1}$)"],
                'b':["b) Change in Total Rainfall (mm day$^{-1}$)","d) Change in Banded Rainfall (mm day$^{-1}$)",\
                "f) Change in Band Frequency (%)","h) Change in Band Intensity (mm day$^{-1}$)"]}

## Define title for each half of the figure
panel_titles = {'a':'1980-2007 v 1951-1979','b':'1994-2007 v 1980-1993'}

## Biggest change from previous version is that instead of explicitly
## enumerating each subplot, now line everything up at the beginning and
## then iterate through each subplot with an identical loop
for half in ['a','b']:
    for row,(precip,contour,title) in enumerate(zip(P_plot_tuples[half],P_contour_tuples[half],\
                                                  subplot_titles[half])):
        ax = plt.subplot(grids[half][row,0])
        axes[half].append(ax)
        ax.pcolormesh(precip, cmap = mypalette, vmin = -axlim, vmax = axlim); 
        ax.contour(contour,1,colors = 'gray')
        ax.set_title(title, fontsize=20, x=0, y = 1.14, horizontalalignment = 'left')
        ax.set_yticklabels(yticklabels)
        ax.set_ylabel('Latitude ($\!^\circ\!$N)', fontsize=12, labelpad=0)
        add_double_calendar_axis()
        mark_time_periods(time_periods, numbers = 'yes', box = 'no')

    ## add title to this half - place right above middle panel in top row
    pos = axes[half][0].get_position() # get the original position 
    xx = (pos.x0+pos.x1)/2
    yy = pos.y1+.07
    plt.figtext(xx,yy,panel_titles[half],horizontalalignment='center',size=24)

/Users/Siwen/anaconda/lib/python3.6/site-packages/matplotlib/text.py:2138: UserWarning: You have used the `textcoords` kwarg, but not the `xytext` kwarg.  This can lead to surprising results.
  warnings.warn("You have used the `textcoords` kwarg, but not "


time: 1.45 s


In [13]:
## SAVE COPIES OF FIGURE TO MANUSCRIPT AND ANALYSIS DIRECTORIES
figure_name = "hov_summary.pdf"

savepath = "/Users/Siwen/Desktop/RDA/Figures/" + figure_name

if os.path.exists(savepath):
    print("Overwriting existing version of file in RDA manuscript directory...")
    os.remove(savepath)
    
plt.savefig(savepath, bbox_inches='tight')
    
copypath = "/Users/Siwen/RDA/Figures/" + figure_name
    
if os.path.exists(copypath):
    print("Overwriting existing version of file in RDA Python directory...")
    os.remove(copypath)
    
copy(savepath,copypath)

plt.savefig(savepath, bbox_inches='tight')

Overwriting existing version of file in RDA manuscript directory...
Overwriting existing version of file in RDA Python directory...
time: 1min 21s
